## Dynamical Network Biomarker (DNB) tools for time series data

---

This code extracts the early warning signal (EWS) reference indicators according to the DNB's idea.
In accordance with the literature [1], this code calculates the indicators by the following steps:
1. normalizing the data.
2. dividing the data into sliding windows.
3. calculating the covariance matrix and the maximum eigenvalue.

This package is available on github and can be used with the following pip code.
```code
pip install https://@github.com/hiroshi-yamashita/dnb-tools
```

In addition to the DNB code, this code contains simulation data where the following branches occur:

1. Toy model with saddle node bifurcation 

2. A harvested population model with herbivores and biomass [2]

3. The spatial type of the above [3]

## References

[1]	M. Oku and K. Aihara, “On the Covariance Matrix of the Stationary Distribution of a Noisy Dynamical System,” Nonlinear Theory and Its Applications, IEICE, 9(2), 166-184, doi:10.1587/nolta.9.166 (2018).

[2] May, Robert M. "Thresholds and breakpoints in ecosystems with a multiplicity of stable states." Nature 269.5628 (1977): 471-477.

[3] van Nes, Egbert H., and Marten Scheffer. "Implications of spatial heterogeneity for catastrophic regime shifts in ecosystems." Ecology 86.7 (2005): 1797-1807.

## Requirements
This packeage need the following:
```code
python >=3.7 
numpy
pandas
matplotlib
sklearn
ruptures
tqdm
```

In [ ]:
# first run this code box to connect to runtime

## Usage


1. Install package


In [ ]:
!pip install git+https://github.com/hiroshi-yamashita/dnb-tools.git

2a. Prepare your data following the format instructions below.
- When using COLABORATORY, upload a csv file. Click on the folder icon on the far left and upload the file from "Upload to Session Storage". 
- You can get example data by following command:

In [ ]:
!dnb_example_timeseries --saddle_node
# See "dnb_example_timeseries --help" for more information.
# !dnb_example_timeseries --help

### Input format

The csv file style is the following:

|Time or Step| feature_name_1 | feature_name_2 | ... | feature_name_n |
|----|----|----|----|----|
|0|$x_1(0)$ |$x_2(0)$ | ... |$x_n(0)$ |
|1|$x_1(1)$ |$x_2(1)$ | ... |$x_n(1)$ |
|...| ... | ... | ... | ... | ... |
|T|$x_1(T)$ |$x_2(T)$ | ... |$x_n(T)$ |

2b. change the following 'filename' variable to the target file name.
(The initial case: the target file name is 'sample.csv')

In [ ]:
input_folder = "input"
filename = 'sample.csv' 

3. Run the analysis script following the instructions below. You can follow either the simple instruction in `Simple to implement` or the detailed instruction in `Step-by-step analysis` section.

4. The output `DNB_*.csv` for subsequent analysis by `dnb_tabular`.


---

## Simple to implement

In [ ]:
!dnb_timeseries --input_path {input_folder} {filename}
# See "dnb_timeseries --help" for more information.

You can use the output `DNB_*.csv` for further analysis by `dnb_tabular`.

- When using COLABORATORY, click on the folder icon on the far left and download the output file. 

---

## Step-by-Step Analysis

When the respective function of dnb_ts is performed, follow the instructions below:

1. import package

In [ ]:
from dnb_tool.timeseries.dnb_ts import EWS_DNB
from dnb_tool.timeseries.dnb_ts import CPD_EWS


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

2. Read data from the csv file

In [ ]:

df = pd.read_csv(f"{input_folder}/{filename}", index_col = 0)
x = df.values[:,1:]
df.head() # look the csv head data 


- If you use the sample data, run the following code:

In [ ]:
# toy model with saddle-node bifurcation
from dnb_tool.datasets import saddle_node_model 
times, x,y =  saddle_node_model.get_data()

# May model [2]
from dnb_tool.datasets import May_model
times, x,y =  May_model.get_data()

# A spatial harvested population model[3]
from dnb_tool.datasets import HP_model
HP_model.overview()
times, x,y =  HP_model.get_data(n = 5,sigma = 0.1)

df = pd.DataFrame(np.c_[times,x])
df.to_csv('sample.csv',index= False)

3. Calculating the EWS and the candidate of the bifurcation point

In [ ]:
# window size of sliding windows
window_size = 1000

# padding type; 
#'valid': no padding, 
#'same': completing the numbers so that the output is centered, 
#'online': completing numbers so that outputs can be calculated online.
padding =  'online' 

# normalization type;
#'straight': not normalized, 
#'std': std of the data become 1, 
#'minmax': the maximum error of data become 1.
#'PCA': Dimensions are compressed by PCA (Output is 10 dimensions).  Use when there are many dimensions.
normalization = 'straight'

ews =  EWS_DNB(x,window_size = window_size,padding = padding, normalization = normalization )

In [ ]:
# 
# Config of change point detection methods; 
#'type': 
#   'peak' : bifucation point assume peak
#   'linear' : Linear prediction
#   'ar' : AR model
#   'Ohtsu' :  01 detection using Ohtsu method
#'dim': order of the target model
cfg = {'type':'ar','dim':2}

# Range to probe the change point from the maximum;
scope_range = 1000

cp = CPD_EWS(ews,cfg = cfg,scope_range = scope_range)

# controll point
control = cp//2

4. Visualizing and save

In [ ]:
# Visualization
fig = plt.figure(figsize =  (8,5))
plt.subplot(2,1,1)
plt.grid()
plt.plot(x)
plt.ylabel('Feature',fontsize = 16)
plt.subplot(2,1,2)
plt.grid()
plt.plot(ews)
plt.scatter(cp,ews[cp],color = 'red',label = 'candidate of bifurcation')
plt.scatter(control,ews[control],color = 'blue',label = 'candidate of control')
plt.legend(fontsize = 16)
plt.ylabel('$\lambda_{max}$(Covariance matrix)',fontsize = 16)
plt.xlabel('Step',fontsize = 16)
fig.tight_layout() 
fig.align_labels()
plt.savefig('EWS_DNB.pdf',bbox_inches='tight')
# Save data 
df_ews = pd.DataFrame(ews,columns =["EWS_DNB"])
df_ews.to_csv('EWS_'+ filename)

In [ ]:
# make DNB tools file
x_cp = x[cp-window_size:cp]
x_control = x[control-window_size:control]

columns = [f'ctrl_{i:06}' for i in range(x_control.shape[0])] + [f'expr_{i:06}' for i in range(x_cp.shape[0])]
index = df.columns[1:]
df_out =  pd.DataFrame(np.r_[x_control,x_cp].T,index = index, columns= columns)
df_out.to_csv('DNB_' + filename)
print(df_out)

You can use the output `DNB_*.csv` for further analysis by `dnb_tabular`.

- When using COLABORATORY, click on the folder icon on the far left and download the output file. 

---

## Author of this code
Yuji Okamoto : yuji.0001@gmail.com

Github: https://github.com/yuji0001/time_series_DNB